In [15]:
from pystac_client import Client
import planetary_computer

catalog = Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

min_lon, min_lat, max_lon, max_lat = -123.5, 37.0, -121.5, 38.5
# Buscar colecciones relevantes por texto
for col in catalog.get_all_collections():
    if "Sentinel-2" in col.title or "Landsat" in col.title:
        print(col.id, col.title)


search = catalog.search(
    collections=["sentinel-2-l2a"],
    bbox=[min_lon, min_lat, max_lon, max_lat],
    datetime="2020-01-01/2020-12-31",
    query={"eo:cloud_cover": {"gt": 20}}  
)


items = list(search.get_items())


landsat-c2-l2 Landsat Collection 2 Level-2
sentinel-2-l2a Sentinel-2 Level-2A
landsat-c2-l1 Landsat Collection 2 Level-1
hls2-s30 Harmonized Landsat Sentinel-2 (HLS) Version 2.0, Sentinel-2 Data
hls2-l30 Harmonized Landsat Sentinel-2 (HLS) Version 2.0, Landsat Data


C:\Users\Amalia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pystac_client\item_search.py:925: FutureWarning: get_items() is deprecated, use items() instead
  warnings.warn(


In [16]:
import datetime

def buscar_pares_entrenamiento(bioma_nombre, coords, n_pares=50, dias_margen=30):
    buffer = 0.1
    bbox = [coords[0]-buffer, coords[1]-buffer, coords[0]+buffer, coords[1]+buffer]
    
    search_cloudy = catalog.search(
        collections=["sentinel-2-l2a"],
        bbox=bbox,
        datetime="2023-01-01/2023-12-31",
        query={"eo:cloud_cover": {"gt": 20, "lt": 60}}, 
        max_items=n_pares
    )
    cloudy_items = list(search_cloudy.items())

    pares = []
    for c_item in cloudy_items:
        # Extraer fecha de la imagen nublada
        fecha_nublada = c_item.datetime
        fecha_inicio = (fecha_nublada - datetime.timedelta(days=dias_margen)).strftime('%Y-%m-%dT%H:%M:%SZ')
        fecha_fin = (fecha_nublada + datetime.timedelta(days=dias_margen)).strftime('%Y-%m-%dT%H:%M:%SZ')

        # B. Buscamos la imagen limpia en ese rango específico de ±30 días
        search_clear = catalog.search(
            collections=["sentinel-2-l2a"],
            bbox=c_item.bbox,
            datetime=f"{fecha_inicio}/{fecha_fin}",
            query={"eo:cloud_cover": {"lt": 5}}, 
            max_items=1
        )
        clear_items = list(search_clear.items())
        
        if clear_items:
            pares.append({
                "bioma": bioma_nombre,
                "id_nublada": c_item.id,
                "id_limpia": clear_items[0].id,
                "nubes_porcentaje": c_item.properties["eo:cloud_cover"],
                "item_nublado": c_item,
                "item_limpio": clear_items[0]
            })
    return pares



In [17]:
import pandas as pd
'''
biomas = {
    "bosque": [-84.0, 10.3],
    "ciudad": [-74.0, 40.7],
    "desierto": [25.0, 25.0],
    "tundra": [68.0, 70.0],
    "sabana": [34.0, -1.3],
    "manglar": [-80.0, 0.5],
    "montaña": [86.9, 27.9],
    "pradera": [-100.0, 44.0],
    "humedal": [-58.0, -34.5],
    "agricultura": [-3.0, 40.0],
    "glaciar": [-72.0, -50.0],
    "volcanico": [-155.0, 19.4],
    "costa": [-77.0, 24.5],
    "isla_tropical": [-60.0, 14.0]
}

'''

biomas = {
    "bosque": [-60.0, -3.0],         
    "ciudad": [2.35, 48.86],          
    "desierto": [-13.0, 23.5],        
    "tundra": [30.0, 69.5],         
    "sabana": [20.0, 9.5],           
    "manglar": [99.0, 8.2],           
    "montaña": [7.0, 46.5],           
    "pradera": [105.0, 49.5],         
    "humedal": [-61.0, -32.0],        
    "agricultura": [-62.5, -31.0],    
    "glaciar": [13.0, 47.0],         
    "volcanico": [-78.6, -0.9],      
    "costa": [-8.9, 41.1],            
    "isla_tropical": [151.7, -16.9],  
    "oceano": [-40.0, 0.0],           
}


dataset_pares = []

for nombre, coords in biomas.items():
    print(f"Buscando pares para {nombre}...")
    dataset_pares.extend(buscar_pares_entrenamiento(nombre, coords, n_pares=100))

df_pares = pd.DataFrame(dataset_pares).drop(columns=['item_nublado', 'item_limpio'])
print(f"\nTotal de pares encontrados: {len(df_pares)}")
display(df_pares.head())


# Ejemplo: Analizar el primer par encontrado
ejemplo = dataset_pares[0]['item_nublado']
props = ejemplo.properties

print(f"Análisis del item: {ejemplo.id}")
print(f"- Nubes totales: {props.get('s2:cloud_shadow_percentage')}%")
print(f"- Nubes Cirrus (finas): {props.get('s2:thin_cirrus_percentage')}%")
print(f"- Sombras: {props.get('s2:cloud_shadow_percentage')}%")

Buscando pares para bosque...
Buscando pares para ciudad...
Buscando pares para desierto...
Buscando pares para tundra...
Buscando pares para sabana...
Buscando pares para manglar...
Buscando pares para montaña...
Buscando pares para pradera...
Buscando pares para humedal...
Buscando pares para agricultura...
Buscando pares para glaciar...
Buscando pares para volcanico...
Buscando pares para costa...
Buscando pares para isla_tropical...
Buscando pares para oceano...

Total de pares encontrados: 813


,bioma,id_nublada,id_limpia,nubes_porcentaje
0,bosque,S2A_MSIL2A_20231218T141711_R010_T20MRB_2023122...,S2B_MSIL2A_20231206T142709_R053_T20MRA_2023120...,23.725198
1,bosque,S2B_MSIL2A_20231206T142709_R053_T20MRB_2024110...,S2B_MSIL2A_20231206T142709_R053_T20MRA_2023120...,34.149659
2,bosque,S2B_MSIL2A_20231206T142709_R053_T20MRB_2023120...,S2B_MSIL2A_20231206T142709_R053_T20MRA_2023120...,34.701788
3,bosque,S2A_MSIL2A_20231111T142711_R053_T20MRB_2024103...,S2B_MSIL2A_20231206T142709_R053_T20MRA_2023120...,23.462209
4,bosque,S2A_MSIL2A_20231111T142711_R053_T20MRB_2023111...,S2B_MSIL2A_20231206T142709_R053_T20MRA_2023120...,23.513471


Análisis del item: S2A_MSIL2A_20231218T141711_R010_T20MRB_20231220T103346
- Nubes totales: 1.708466%
- Nubes Cirrus (finas): 11.660299%
- Sombras: 1.708466%


In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import rioxarray
import numpy as np
import torchvision.transforms.functional as TF
import random

class CloudSegmentationDataset(Dataset):
    def __init__(self, df_pares, patch_size=256, train=True):
        self.df = df_pares
        self.patch_size = patch_size
        self.train = train
        self.bands = ["B04", "B03", "B02", "B08", "B11", "B12"]

    def __len__(self):
        return len(self.df)

    def transform(self, image, mask, target):
        # Image y Target son tensores (C, H, W). Mask es (1, H, W)
        if self.train:
            # Flip horizontal
            if random.random() > 0.5:
                image = TF.hflip(image)
                mask = TF.hflip(mask)
                target = TF.hflip(target)

            # Rotaciones de 90 grados
            angle = random.choice([0, 90, 180, 270])
            if angle != 0:
                image = TF.rotate(image, angle)
                mask = TF.rotate(mask, angle)
                target = TF.rotate(target, angle)

        return image, mask, target

    def _get_patch(self, item, is_mask=False):
        signed_item = planetary_computer.sign(item)
        if is_mask:
            url = signed_item.assets["SCL"].href
            da = rioxarray.open_rasterio(url)
            patch = da.isel(y=slice(0, self.patch_size), x=slice(0, self.patch_size)).values
            mask = np.isin(patch, [3, 8, 9, 10]).astype(np.float32)
            return torch.from_numpy(mask) # Retorna (1, H, W) o (H, W)
        else:
            band_data = []
            for b in self.bands:
                url = signed_item.assets[b].href
                da = rioxarray.open_rasterio(url)
                patch = da.isel(y=slice(0, self.patch_size), x=slice(0, self.patch_size)).values
                # Normalización robusta por banda: (x - min) / (max - min) aproximado
                # Sentinel-2 escala 0-10000, pero raramente pasa de 4000 en reflectancia
                patch = np.clip(patch / 4000.0, 0, 1)
                band_data.append(patch)

            img = np.concatenate(band_data, axis=0).astype(np.float32)
            return torch.from_numpy(img)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        img_input = self._get_patch(row['item_nublado'], is_mask=False)
        mask = self._get_patch(row['item_nublado'], is_mask=True)
        img_target = self._get_patch(row['item_limpio'], is_mask=False)

        # Aplicar aumentos de datos de forma consistente a los 3 elementos
        img_input, mask, img_target = self.transform(img_input, mask, img_target)

        return {
            "input": img_input,   # Para Modelo A y B
            "mask": mask,         # Target para Modelo A, Input para Modelo B
            "target": img_target  # Target para Modelo B
        }

# Crear el DataLoader
train_ds = CloudSegmentationDataset(df_pares)
train_loader = DataLoader(train_ds, batch_size=8, shuffle=True)

KeyboardInterrupt: 

In [19]:
# Verifica que cada fila tenga un ID distinto pero cercano
print(df_pares[['id_nublada', 'id_limpia', 'nubes_porcentaje']].head(11))

# Cuenta cuántos pares tienes por cada tipo de bioma
print(df_pares['bioma'].value_counts())

                                           id_nublada  \
0   S2A_MSIL2A_20231218T141711_R010_T20MRB_2023122...   
1   S2B_MSIL2A_20231206T142709_R053_T20MRB_2024110...   
2   S2B_MSIL2A_20231206T142709_R053_T20MRB_2023120...   
3   S2A_MSIL2A_20231111T142711_R053_T20MRB_2024103...   
4   S2A_MSIL2A_20231111T142711_R053_T20MRB_2023111...   
5   S2A_MSIL2A_20231029T141711_R010_T20MRB_2023102...   
6   S2A_MSIL2A_20231019T141711_R010_T20MRB_2024092...   
7   S2A_MSIL2A_20231019T141711_R010_T20MRB_2023101...   
8   S2B_MSIL2A_20231017T142719_R053_T20MRB_2024110...   
9   S2B_MSIL2A_20231017T142719_R053_T20MRB_2023101...   
10  S2A_MSIL2A_20230929T141711_R010_T20MRB_2023092...   

                                            id_limpia  nubes_porcentaje  
0   S2B_MSIL2A_20231206T142709_R053_T20MRA_2023120...         23.725198  
1   S2B_MSIL2A_20231206T142709_R053_T20MRA_2023120...         34.149659  
2   S2B_MSIL2A_20231206T142709_R053_T20MRA_2023120...         34.701788  
3   S2B_MSIL2A_2023

## Guardar pares y parches

Funciones para guardar metadata de pares y (opcional) descargar parches centrados para uso en entrenamiento.

In [20]:
import os
import numpy as np
import rioxarray
import planetary_computer
import pandas as pd

def save_pairs_metadata(pares, out_csv="pares_metadata.csv"):
    rows = []
    for p in pares:
        try:
            n = planetary_computer.sign(p["item_nublado"])
            l = planetary_computer.sign(p["item_limpio"])
        except Exception:
            n = p.get("item_nublado")
            l = p.get("item_limpio")
        rows.append({
            "bioma": p.get("bioma"),
            "id_nublada": p.get("id_nublada"),
            "id_limpia": p.get("id_limpia"),
            "nubes_porcentaje": p.get("nubes_porcentaje"),
            "nublado_B04": n.assets.get("B04").href if n and "B04" in getattr(n, 'assets', {}) else None,
            "nublado_SCL": n.assets.get("SCL").href if n and "SCL" in getattr(n, 'assets', {}) else None,
            "limpio_B04": l.assets.get("B04").href if l and "B04" in getattr(l, 'assets', {}) else None,
        })
    pd.DataFrame(rows).to_csv(out_csv, index=False)
    print(f"Metadata guardada en {out_csv}")

def save_center_patch(item, out_path, bands=["B04","B03","B02"], patch_size=256):

    planetary_computer.sign_inplace(item)  

    with rioxarray.open_rasterio(item.assets[bands[0]].href) as ref:
        cy, cx = ref.shape[1] // 2, ref.shape[2] // 2

    band_arrays = []

    for b in bands:
        with rioxarray.open_rasterio(item.assets[b].href) as da:
            patch = da.isel(
                y=slice(cy, cy + patch_size),
                x=slice(cx, cx + patch_size)
            ).values
            band_arrays.append(patch)

    arr = np.concatenate(band_arrays, axis=0).astype(np.float32)
    np.save(out_path, arr)

def download_all_pairs(pares, out_root="pairs_patches", patch_size=256, max_pairs=None, start_from=0):
    os.makedirs(out_root, exist_ok=True)
    end_at = len(pares) if max_pairs is None else min(len(pares), start_from + max_pairs)
    for i in range(start_from, end_at):
        p = pares[i]
        name = f"{i}_{p.get('bioma','')}_{p.get('id_nublada','')}".replace(' ', '_')
        dir_out = os.path.join(out_root, name)
        # Verificar si ya existe (ya descargado)
        if os.path.exists(dir_out) and os.path.isfile(os.path.join(dir_out, "input.npy")):
            print(f"Par {i} ya descargado, saltando...")
            continue
        os.makedirs(dir_out, exist_ok=True)
        try:
            save_center_patch(p["item_nublado"], os.path.join(dir_out, "input.npy"), patch_size=patch_size)
            save_center_patch(p["item_limpio"], os.path.join(dir_out, "target.npy"), patch_size=patch_size)
            try:
                save_center_patch(p["item_nublado"], os.path.join(dir_out, "scl.npy"), bands=["SCL"], patch_size=patch_size)
            except Exception:
                pass
            print(f"Par {i} descargado exitosamente.")
        except Exception as e:
            print(f"Error guardando par {i}: {e}")

# Ejemplo: guardar metadata y descargar 20 pares (descomenta para ejecutar)
save_pairs_metadata(dataset_pares, out_csv="pares_metadata_v2.csv")
# Para continuar desde donde se quedó, encuentra el último índice descargado
# Por ejemplo, si se quedó en 50, usa start_from=50
download_all_pairs(dataset_pares, out_root="pairs_patches_sample_v2", patch_size=256, max_pairs=800, start_from=107)

Metadata guardada en pares_metadata_v2.csv
Par 107 ya descargado, saltando...
Par 108 descargado exitosamente.
Par 109 descargado exitosamente.
Par 110 descargado exitosamente.
Par 111 descargado exitosamente.
Par 112 descargado exitosamente.
Par 113 descargado exitosamente.
Par 114 descargado exitosamente.
Par 115 descargado exitosamente.
Error guardando par 116: HTTP response code: 403
Error guardando par 117: HTTP response code: 403
Error guardando par 118: HTTP response code: 403
Error guardando par 119: HTTP response code: 403
Error guardando par 120: HTTP response code: 403
Error guardando par 121: HTTP response code: 403
Error guardando par 122: HTTP response code: 403
Error guardando par 123: HTTP response code: 403
Error guardando par 124: HTTP response code: 403
Error guardando par 125: HTTP response code: 403
Error guardando par 126: HTTP response code: 403
Error guardando par 127: HTTP response code: 403
Error guardando par 128: HTTP response code: 403
Error guardando par 1